# speed

> Speed modules for benchmarking

In [ ]:
#| default_exp speed

In [ ]:
#| export
from __future__ import annotations
import math, time
from dataclasses import dataclass, asdict
from typing import Dict, Sequence, List, Mapping

import numpy as np
import torch
import torch.nn as nn
from torch.utils.benchmark import Timer


from fasterbench.core import _device_ctx, _sync

/home/HubensN/miniconda3/envs/fasterai20/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
#| export
@dataclass
class SpeedMetrics:
    p50_ms: float
    p90_ms: float
    p99_ms: float
    mean_ms: float
    std_ms: float
    throughput_s: float

    def as_dict(self):
        return asdict(self)


#| export
def _stats(lat_ms: np.ndarray, batch: int) -> Mapping[str, float]:
    return {
        "p50_ms": float(np.percentile(lat_ms, 50)),
        "p90_ms": float(np.percentile(lat_ms, 90)),
        "p99_ms": float(np.percentile(lat_ms, 99)),
        "mean_ms": float(lat_ms.mean()),
        "std_ms": float(lat_ms.std(ddof=1)),
        "throughput_s": batch * 1000.0 / float(lat_ms.mean()),
    }

#| export
def _forward_latencies(
    model: nn.Module,
    sample: torch.Tensor,
    *,
    device: str | torch.device = "cpu",
    warmup: int = 20,
    steps: int = 100,
    use_torch_timer: bool | None = None,
) -> np.ndarray:
    """
    Collect `steps` forward-pass latencies (ms).  Uses the low-overhead
    ``torch.utils.benchmark.Timer`` on CPU and ``torch.cuda.Event`` on GPU.
    """
    if use_torch_timer is None:
        use_torch_timer = torch.device(device).type == "cpu"

    with _device_ctx(device) as dev:
        model.eval().to(dev)
        sample = sample.to(dev, non_blocking=True)

        # ─ warm-up
        for _ in range(warmup):
            model(sample)
        _sync(dev)

        lat: List[float] = []
        if use_torch_timer and dev.type == "cpu":
            t = Timer(stmt="model(x)", globals={"model": model, "x": sample})
            m = t.blocked_autorange(min_run_time=0.3)
            per_iter = (np.asarray(m.raw_times) / m.number_per_run) * 1e3  # ns→ms
            if per_iter.size < steps:
                per_iter = np.resize(per_iter, steps)   # pad so len==steps
            lat = per_iter.tolist()
        elif dev.type == "cuda":
            start_evt, end_evt = torch.cuda.Event(True), torch.cuda.Event(True)
            for _ in range(steps):
                start_evt.record()
                model(sample)
                end_evt.record()
                _sync(dev)
                lat.append(start_evt.elapsed_time(end_evt))  # ms
        else:  # plain time.perf_counter fallback
            for _ in range(steps):
                t0 = time.perf_counter()
                model(sample)
                lat.append((time.perf_counter() - t0) * 1e3)
        return np.asarray(lat, dtype=np.float32)

#| export
@torch.inference_mode()
def compute_speed(model: nn.Module,
                  sample: torch.Tensor,
                  *,
                  device: str | torch.device = "cpu",
                  warmup: int = 20,
                  steps: int = 100) -> SpeedMetrics:
    """
    Measure latency/throughput on *one* device.  Returns a `SpeedMetrics`.
    """
    with _device_ctx(device) as dev:
        lat = _forward_latencies(
            model, sample, device=dev, warmup=warmup, steps=steps
        )
        s = _stats(lat, sample.size(0))
        return SpeedMetrics(**s)

#| export
def compute_speed_multi(model: nn.Module,
                        sample: torch.Tensor,
                        *,
                        devices: Sequence[str | torch.device] | None = None,
                        **kwargs) -> Dict[str, SpeedMetrics]:
    """
    Convenience wrapper: returns ``dict[str, SpeedMetrics]`` keyed by device.
    Defaults to both CPU *and* CUDA if a GPU is available.
    """
    if devices is None:
        devices = ["cpu"]
        if torch.cuda.is_available():
            devices.append("cuda")

    out: Dict[str, SpeedMetrics] = {}
    for d in devices:
        try:
            out[str(d)] = compute_speed(model, sample, device=d, **kwargs)
        except RuntimeError:
            out[str(d)] = SpeedMetrics(*[math.nan] * 6)
    return out

#| export
def sweep_threads(model: nn.Module,
                  sample: torch.Tensor,
                  thread_counts: Sequence[int] = (1, 2, 4, 8),
                  *,
                  warmup: int = 20,
                  steps: int = 100) -> "torch.Tensor":
    """
    Return a *pandas-compatible* list-of-dicts; each row contains latency
    stats for a different ``torch.set_num_threads(n)``.
    """
    rows = []
    for n in thread_counts:
        torch.set_num_threads(n)
        lat = _forward_latencies(
            model, sample,
            device="cpu",
            warmup=warmup,
            steps=steps,
            use_torch_timer=True,
        )
        row = {"threads": n, **_stats(lat, sample.size(0))}
        rows.append(row)
    return rows   # user can `pd.DataFrame(rows)` if they wish

#| export
def sweep_latency(model: nn.Module,
                  shapes: Sequence[Sequence[int]],
                  *,
                  device: str | torch.device = "cuda",
                  warmup: int = 20,
                  steps: int = 100) -> "torch.Tensor":
    """
    Sweep different *input shapes* on the same device.
    Useful for CNNs/ViTs where latency scales with resolution.
    """
    rows = []
    for shape in shapes:
        dummy = torch.empty(*shape)
        lat = _forward_latencies(
            model, dummy,
            device=device,
            warmup=warmup,
            steps=steps,
        )
        row = {"shape": "×".join(map(str, shape)), **_stats(lat, shape[0])}
        rows.append(row)
    return rows
